In [34]:
import os
import datetime

In [35]:
DOWNLOAD_FOLDER = 'D:\\letöltések\\weather_data'
os.chdir("C:\\Program Files (x86)\\Google\\Chrome\\Application\\")

In [36]:
if len(os.listdir(DOWNLOAD_FOLDER)) == 0:
    base_command = '.\\chrome.exe "http://www.amsz.hu/ws/includes/export.php?id=4&save=xls&year={}&month={}&day={}"'
    base_command.format(2018,5,20)
    current_day = datetime.date(2008, 3, 2)  #year, month, day
    today = datetime.date.today()
    d_one_day = datetime.timedelta(days=1)
    while current_day < today:
        command = base_command.format(current_day.year, current_day.month, current_day.day)
        os.system(command)
        current_day += d_one_day